# GPT

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# Устанавливаем API ключ и базовый URL
openai_api_key = "EMPTY"  # Ключ не требуется, так как используется локальный сервер
openai_api_base = "http://localhost:8071/v1"  # Адрес вашего локального API сервера

# Создаем экземпляр ChatOpenAI с модифицированными параметрами
chat = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    model_name="Qwen/Qwen2.5-3B-Instruct-GPTQ-Int4",  # Указываем имя модели
    max_tokens=5000,  # Ограничиваем количество токенов в ответе
    temperature=0.1  # Устанавливаем температуру (от 0 до 1, где 0 - детерминированный ответ, 1 - более случайный)
)

# Системный промпт, который задает контекст для модели
system_prompt = SystemMessage(content="Вы полезный помощник, который отвечает на вопросы на русском языке. Ваши ответы должны быть четкими, информативными и полезными.")

# Вопрос пользователя
question = "Как работает нейронная сеть? Ответь кратко"

# Формируем итоговый промпт
messages = [system_prompt, HumanMessage(content=question)]

# Выводим итоговый промпт
print("Итоговый промпт, подаваемый на вход модели:")
for message in messages:
    print(f"{message.type}: {message.content}")

Итоговый промпт, подаваемый на вход модели:
system: Вы полезный помощник, который отвечает на вопросы на русском языке. Ваши ответы должны быть четкими, информативными и полезными.
human: Как работает нейронная сеть? Ответь кратко


In [12]:
# Отправляем запрос модели
response = chat(messages)

# Выводим ответ
print("Ответ модели:", response.content)

Ответ модели: Нейронная сеть - это система, которая имитирует работу человеческого мозга. Она состоит из множества нейронов (или узлов), которые обмениваются сигналами. Эти сигналы формируют слои, которые обрабатывают входящую информацию. Входные данные проходят через эти слои, где они обрабатываются и преобразуются в выходные данные. Алгоритмы обучения нейронных сетей используются для настройки этих нейронов, чтобы они могли лучше обрабатывать входные данные.


In [13]:
response

AIMessage(content='Нейронная сеть - это система, которая имитирует работу человеческого мозга. Она состоит из множества нейронов (или узлов), которые обмениваются сигналами. Эти сигналы формируют слои, которые обрабатывают входящую информацию. Входные данные проходят через эти слои, где они обрабатываются и преобразуются в выходные данные. Алгоритмы обучения нейронных сетей используются для настройки этих нейронов, чтобы они могли лучше обрабатывать входные данные.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 69, 'total_tokens': 199, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-3B-Instruct-GPTQ-Int4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3b18c959-8dcc-48fc-b6be-5fb73a210ee4-0')

# Эмбеддер

In [8]:
from langchain.embeddings.base import Embeddings
import requests
from typing import List

class CustomEmbedder(Embeddings):
    def __init__(self, embedder_url: str):
        self.embedder_url = embedder_url

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Получает эмбеддинги для списка текстов."""
        response = requests.post(
            self.embedder_url,
            json={"inputs": texts}
        )
        if response.status_code == 200:
            return response.json()[0]
        else:
            raise Exception(f"Ошибка: {response.status_code}, {response.text}")

    def embed_query(self, text: str) -> List[float]:
        """Получает эмбеддинг для одного текста."""
        return self.embed_documents([text])

# Укажите URL вашего эмбеддера
EMBEDDER_URL = "http://localhost:8080/embed"

# Создаем экземпляр кастомного эмбеддера
custom_embedder = CustomEmbedder(embedder_url=EMBEDDER_URL)

# Пример использования
texts = [
    "LangChain — это мощный инструмент для работы с языковыми моделями."
]

# Получаем эмбеддинги для списка текстов
embeddings = custom_embedder.embed_documents(texts)
print("Эмбеддинги для документов:", embeddings)

# Получаем эмбеддинг для одного текста
query_embedding = custom_embedder.embed_query("Как работает LangChain?")
print("Эмбеддинг для запроса:", query_embedding)

Эмбеддинги для документов: [0.023204839, 9.58146e-05, 0.012926118, -0.048856515, 0.03164843, -0.021921253, -0.0096970955, 0.0717204, 0.018712286, -0.045206316, 0.02426781, 0.04239847, -0.0014164579, 0.01731839, -0.03257101, -0.011151158, -0.05651792, 0.008368383, -0.033714205, -0.013648136, 0.04841528, 0.0075861975, -0.013868752, -0.01600472, -0.032189947, -0.017859904, -0.035198353, -0.058322966, 0.015914468, -0.050180215, 0.006718774, 0.013437548, 0.017739568, -0.0247291, -0.01737856, 0.04677069, 0.023926856, 0.02825896, -0.021379739, 0.060809914, -0.0020983634, 0.05748061, 0.0016333138, -0.01347766, -0.008443593, 0.009125498, 0.059807114, -0.03261112, 0.0063176528, -0.008543873, 0.02198142, 0.015794132, -0.02689515, -0.012063708, -0.038066365, 0.021139067, -0.035318688, 0.024528537, -0.04604867, 0.037564963, -0.021820972, -0.008172836, 0.030725854, -0.015663767, -0.027877897, 0.02198142, 0.029121371, 0.039149392, -0.045406878, -0.0137785, -0.012555081, 0.022502877, -0.03439611, -0.0

# Реранкер

In [9]:
from typing import List
import requests
from langchain.schema import Document

class CustomReranker:
    def __init__(self, reranker_url: str):
        self.reranker_url = reranker_url

    def rerank(self, query: str, documents: List[Document]) -> List[Document]:
        """
        Пересчитывает релевантность документов на основе запроса.
        """
        # Преобразуем документы в текстовый формат
        texts = [doc.page_content for doc in documents]

        # Отправляем запрос к реранкеру
        response = requests.post(
            self.reranker_url,
            json={"query": query, "texts": texts}  # Используем "texts" вместо "documents"
        )

        if response.status_code != 200:
            raise Exception(f"Ошибка: {response.status_code}, {response.text}")

        # Получаем результаты реранкинга
        results = response.json()  # Это список словарей [{"index": ..., "score": ...}]
        reranked_docs = []

        # Сортируем результаты по убыванию оценки
        results.sort(key=lambda x: x["score"], reverse=True)

        # Сопоставляем результаты с документами
        for result in results:
            index = result["index"]
            score = result["score"]
            doc = documents[index]
            doc.metadata["score"] = score
            reranked_docs.append(doc)

        return reranked_docs

# Укажите URL вашего реранкера
RERANKER_URL = "http://localhost:8081/rerank"

# Создаем экземпляр кастомного реранкера
custom_reranker = CustomReranker(reranker_url=RERANKER_URL)

# Пример использования
query = "Что такое LangChain?"
documents = [
    Document(page_content="Lang Chain предоставляет модели", metadata={"source": "doc3"}),
    Document(page_content="LangChain — это фреймворк для работы с языковыми моделями.", metadata={"source": "doc1"}),
    Document(page_content="LangChain позволяет создавать цепочки вызовов языковых моделей.", metadata={"source": "doc2"}),
    Document(page_content="Hugging Face предоставляет модели для NLP.", metadata={"source": "doc3"}),
]

# Применяем реранкер
reranked_docs = custom_reranker.rerank(query, documents)

# Выводим результаты
for doc in reranked_docs:
    print(f"Документ: {doc.page_content}, Релевантность: {doc.metadata['score']}")

Документ: LangChain — это фреймворк для работы с языковыми моделями., Релевантность: 0.99984646
Документ: LangChain позволяет создавать цепочки вызовов языковых моделей., Релевантность: 0.94218844
Документ: Lang Chain предоставляет модели, Релевантность: 0.08021325
Документ: Hugging Face предоставляет модели для NLP., Релевантность: 1.683241e-05


# VQA

In [55]:
import base64
from openai import OpenAI

def encode_image_to_base64(image_path):
    """Кодирует изображение в base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def ask_model_with_image(image_path, prompt, temperature=0.7, max_tokens=150):
    """
    Запрашивает ответ у модели, используя изображение и текстовый промпт.

    :param image_path: Путь к локальному изображению.
    :param prompt: Текстовый промпт для модели.
    :param temperature: Температура для контроля случайности ответа (по умолчанию 0.7).
    :param max_tokens: Максимальное количество токенов в ответе (по умолчанию 150).
    :return: Ответ модели.
    """
    # Закодируйте изображение в base64
    base64_image = encode_image_to_base64(image_path)

    # Настройка клиента OpenAI
    openai_api_key = "EMPTY"
    openai_api_base = "http://localhost:8089/v1"
    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )

    # Создание запроса с локальным изображением
    chat_response = client.chat.completions.create(
        model="Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        # Передаем закодированное изображение
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    },
                },
            ],
        }],
        temperature=temperature,  # Указываем температуру
        max_tokens=max_tokens,    # Ограничиваем ответ по токенам
    )

    # Возвращаем ответ модели
    return chat_response.choices[0].message.content

# Пример использования функции
image_path = "test_content/1.jpeg"  # Укажите путь к изображению
prompt = "Укажи все фамилии что фигурируют в документе а также кто эти люди если известно"
response = ask_model_with_image(image_path, prompt, temperature=0.7, max_tokens=200)
print("Ответ модели:", response)

Ответ модели: В документе упоминаются следующие фамилии:

1. Сюрин А.В. - Генеральный директор ООО «БРИЗ Строительные машины».

Не указано, кто именно является Аудитором, но в тексте документа упоминается, что ООО «Транссиб-Аудит» является партнером в области оказания аудиторских услуг в течение 5 лет.


In [56]:
# Пример использования функции
image_path = "test_content/1.jpeg"  # Укажите путь к изображению
prompt = "Скажи адрес компании согласно документу"
response = ask_model_with_image(image_path, prompt, temperature=0.7, max_tokens=200)
print("Ответ модели:", response)

Ответ модели: Адрес компании согласно документу:

Ул. Руставели, д. 14, стр. 6,
127254, Москва, Россия


In [66]:
# Пример использования функции
image_path = "test_content/1.jpeg"  # Укажите путь к изображению
prompt = "Скажи адрес компании согласно документу. Не пиши ничего лишнего. Нужен только ответ. ОТВЕТ:"
response = ask_model_with_image(image_path, prompt, temperature=0.1, max_tokens=200)
print(response)

Руставели, д. 14, стр. 6
